In [2]:
# Import Library
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
# from sklearn.metrics import classification_report, confusion_matrix
# import matplotlib.pyplot as plt


In [ ]:
# Path Dataset
train_dir = '../input/fruits-fresh-and-rotten-for-classification/dataset/train'
test_dir = '../input/fruits-fresh-and-rotten-for-classification/dataset/test'

# Kelas yang Dipilih: Hanya Apel dan Pisang
selected_classes = ['freshapples', 'rottenapples', 'freshbanana', 'rottenbanana']

# Augmentasi Data untuk Training dan Normalisasi untuk Testing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,  # Kurangi rotasi untuk mengurangi distorsi berlebihan
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.8, 1.2),
    zoom_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    validation_split=0.2  # 20% data validasi
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Data Training dan Validation
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    classes=selected_classes,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    classes=selected_classes,
    subset='validation',
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# Data Testing
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    classes=selected_classes,
    class_mode='categorical',
    shuffle=False
)

# Debug Data Generator
print(f"Jumlah sampel dalam training set: {train_gen.samples}")
print(f"Jumlah sampel dalam validation set: {val_gen.samples}")
print(f"Jumlah sampel dalam testing set: {test_gen.samples}")
print("Classes Detected:", train_gen.class_indices)


In [ ]:
# Definisi Model ResNet-50 dengan Pretrained Weights
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Model Sekuensial
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Tambahkan neuron lebih banyak
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax', kernel_regularizer=l2(0.001))  # Klasifikasi 4 kelas
])

# Kompilasi Model
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
# Early Stopping dan ReduceLROnPlateau
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Melatih Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[early_stop, lr_scheduler]
)


In [ ]:
# Evaluasi pada Data Training
train_loss, train_acc = model.evaluate(train_gen)
print(f"Training Accuracy: {train_acc * 100:.2f}%, Loss: {train_loss:.4f}")

# Visualisasi Kurva Akurasi dan Loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.show()

# Classification Report untuk Training
train_predictions = np.argmax(model.predict(train_gen), axis=1)
true_train_labels = train_gen.classes
train_class_names = list(train_gen.class_indices.keys())

print("Classification Report (Training):")
print(classification_report(true_train_labels, train_predictions, target_names=train_class_names))

# Confusion Matrix Berwarna
conf_matrix_train = confusion_matrix(true_train_labels, train_predictions)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix_train, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Training)')
plt.colorbar()
tick_marks = np.arange(len(train_class_names))
plt.xticks(tick_marks, train_class_names, rotation=45)
plt.yticks(tick_marks, train_class_names)

# Tambahkan angka ke dalam box Confusion Matrix
for i in range(conf_matrix_train.shape[0]):
    for j in range(conf_matrix_train.shape[1]):
        plt.text(j, i, format(conf_matrix_train[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix_train[i, j] > conf_matrix_train.max() / 2. else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Perhitungan mAP dan IoU untuk Training
from sklearn.metrics import average_precision_score, jaccard_score

# Convert ke One-Hot Encoding untuk mAP
true_train_labels_one_hot = np.eye(len(train_class_names))[true_train_labels]
train_predictions_prob = model.predict(train_gen)

# Mean Average Precision (mAP)
mAP_train = average_precision_score(true_train_labels_one_hot, train_predictions_prob, average="macro")
print(f"Mean Average Precision (Training): {mAP_train:.4f}")

# Intersection over Union (IoU)
IoU_train = jaccard_score(true_train_labels, train_predictions, average="macro")
print(f"Intersection over Union (IoU) (Training): {IoU_train:.4f}")


In [ ]:
# Evaluasi pada Data Testing
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc * 100:.2f}%, Loss: {test_loss:.4f}")

# Classification Report untuk Testing
test_predictions = np.argmax(model.predict(test_gen), axis=1)
true_test_labels = test_gen.classes
test_class_names = list(test_gen.class_indices.keys())

print("Classification Report (Testing):")
print(classification_report(true_test_labels, test_predictions, target_names=test_class_names))

# Confusion Matrix Berwarna
conf_matrix_test = confusion_matrix(true_test_labels, test_predictions)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix_test, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Testing)')
plt.colorbar()
tick_marks = np.arange(len(test_class_names))
plt.xticks(tick_marks, test_class_names, rotation=45)
plt.yticks(tick_marks, test_class_names)

# Tambahkan angka ke dalam box Confusion Matrix
for i in range(conf_matrix_test.shape[0]):
    for j in range(conf_matrix_test.shape[1]):
        plt.text(j, i, format(conf_matrix_test[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix_test[i, j] > conf_matrix_test.max() / 2. else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Perhitungan mAP dan IoU untuk Testing
# Convert ke One-Hot Encoding untuk mAP
true_test_labels_one_hot = np.eye(len(test_class_names))[true_test_labels]
test_predictions_prob = model.predict(test_gen)

# Mean Average Precision (mAP)
mAP_test = average_precision_score(true_test_labels_one_hot, test_predictions_prob, average="macro")
print(f"Mean Average Precision (Testing): {mAP_test:.4f}")

# Intersection over Union (IoU)
IoU_test = jaccard_score(true_test_labels, test_predictions, average="macro")
print(f"Intersection over Union (IoU) (Testing): {IoU_test:.4f}")


In [ ]:
# Menyimpan model dalam format .h5
model.save('/kaggle/working/model_viskom.h5')
